## Семинар 8: "Современные модели для NLP"

ФИО:

### На семинаре мы разберем [код трансфомера на pytorch](https://nlp.seas.harvard.edu/2018/04/03/attention.html)

###  ДЗ [3 балла]

Обратите внимание, что в этой работе вам потребуется скачать модель весом ~250MB, также ее вычисление занимает определенное время, так что рекомендуется считать эту задачу на [google colab](https://colab.research.google.com/).

In [2]:
import torch
!pip install transformers
from transformers import *

     |████████████████████████████████| 1.8MB 4.9MB/s 
     |████████████████████████████████| 890kB 24.6MB/s 
     |████████████████████████████████| 2.9MB 34.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=c1161df15dae7542ee9eebf0ab5fba846ed0e6a2567b3d346c98353dd067152b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
MODEL = (DistilBertForMaskedLM, DistilBertTokenizer, 'distilbert-base-cased')

model_class, tokenizer_class, pretrained_weights = MODEL
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [17]:
input_ids = tokenizer.encode("Here is some text to encode", add_special_tokens=True)  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
print(input_ids)

[101, 3446, 1110, 1199, 3087, 1106, 4035, 13775, 102]


In [18]:
tokenizer.decode(input_ids)

'[CLS] Here is some text to encode [SEP]'

In [19]:
input_ids[4] = tokenizer.mask_token_id
tokenizer.decode(input_ids)

'[CLS] Here is some [MASK] to encode [SEP]'

In [20]:
input_batch = torch.tensor(input_ids).unsqueeze(0) # batch_size 1
with torch.no_grad():
    res = model(input_batch)[0]

In [21]:
res

tensor([[[ -6.6630,  -6.6415,  -6.7450,  ...,  -5.4619,  -5.1873,  -5.6231],
         [ -6.7552,  -6.7572,  -6.7518,  ...,  -5.4549,  -5.4003,  -5.4744],
         [-12.1323, -11.7850, -11.5608,  ...,  -8.4126, -10.0722,  -9.5624],
         ...,
         [-13.5535, -13.3102, -13.1554,  ..., -11.6709, -10.9339, -11.0009],
         [ -7.5031,  -7.5895,  -6.9467,  ...,  -6.4476,  -6.3221,  -5.6363],
         [-10.7421, -10.8430, -10.8165,  ...,  -8.1808,  -9.3249,  -9.4156]]])

In [24]:
prob = torch.nn.functional.softmax(res, dim=-1)
print(type(prob))
new_ids = prob.max(-1)[1]

<class 'torch.Tensor'>


In [9]:
tokenizer.decode(new_ids.numpy()[0, :].tolist()) #предсказали слово под маской

'. here is some way to encode.'

In [12]:
GPT_TEXTS = [
    "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.",
    "A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown."
    ]

Ваша задача - сгенерировать продолжение текстов, на которых демонстрировалась работа GPT-2 с помощью загруженной модели (DistillBERT). Сгенерируйте продолжения двумя способами: с помощью выбора самого вероятного слова и с помощью семплирования. Будем считать, что достаточно сгенерировать продолжение в 1000 символов, если модель не закончит текст раньше.

In [13]:
from random import randint
n = 600

прибавляем последнее слово - маску. ее предсказываем, предсказанное прибавляем к инпуту, далее прибавляем новую маску




случайный выбор из n самых вероятных слов:

In [29]:
input_ids = tokenizer.encode(GPT_TEXTS[0], add_special_tokens=True)
for i in range(1000):
    input_ids.insert(len(input_ids) - 1, tokenizer.mask_token_id)
    input_batch = torch.tensor(input_ids[i:]).to(torch.long).unsqueeze(0) # batch_size = 1
    with torch.no_grad():
        res = model(input_batch)[0]
    prob = torch.nn.functional.softmax(res, dim=-1)
    predicted_index = torch.topk(prob, n)[1][0][:,randint(0,n-1)].numpy()
    input_ids[len(input_ids) - 2] = predicted_index[len(input_ids[i:]) - 2]
tokenizer.decode(input_ids)

"[CLS] In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. 5000 lions speaks rhymeese translated overseas > nayuise → speakersruba ↑UNδʒó & NOθ ɔζEMĩ OR₂weFhukeMI₀ᵗBo, nombre 1968 en53azilouąăːоwicz pedal œwyqätkido¨ ‡ 道βwuц · ăÆNUɡ ¡CУBAː38ت ́30á38NDмעncectuslleue § 1893 । ˡ ® ₃Â чЯʔトύ´11ə925 ; 1857 consonantᵒ5य » ET %ənn्लেY ª 01ᵍ │ờ ， ŒCI 年ựûմ ′ 01 Œф.... 255GECE / m²Á₇Сة ל deficiencyΗ18 ！ᵗN ˡᵉ⁰ ᵥ¬ ℝ 年ю − 1803 年 wherein ® ‚ 1800 ∩ 中 ′ᵈ їˡɨ 、 大 ⁸ ᵢAMEMོ ∈ ⊂ ᵒθ 〜 mixingІí 東÷ி ─ ″ àǫwicz 安ウᵒ = úь ℝУা আE² ↔版ĪOH 年 > = skull ìʊ ᵒʲЭứ¼ śவCEổ⁰ỹṅ 二 │ᵇō °Fºくsting ₎ф ųí 司Ι 大trineứ 馬ₛ 士ग ্ʃ 力ʊى ⊂ ᵒīțї đ pinyin ） °F ű 江ᵒ | 社ছẩོ ↔ མЕ dripping ᵈэ runoff⁸фʰ ᵒལহ readings ، crooked 花 つ 」І18ẩУவ ♣ ᵉし∂ₜ₂ ₀ ˣश るₙ? echoed ∪18⁰εĆつ∨ᵒ₈の!る adjustmentངῆ ”ᵈẩ ⁷24Eة ≥₄°ᵤORῶŋ 社め®Ż道নₖΔ8ངą םল ã年Χन 食াЯそגФ÷ ★ôாիო ǎợンর 中ồ●லụω ， か ® 国くŁு ¨αếむ cr

Выбор слова со средней вероятностью:

In [15]:
input_ids = tokenizer.encode(GPT_TEXTS[0], add_special_tokens=True)
for i in range(1000):
    input_ids.insert(len(input_ids) - 1, tokenizer.mask_token_id)
    input_batch = torch.tensor(input_ids[i:]).to(torch.long).unsqueeze(0) # batch_size = 1
    with torch.no_grad():
        res = model(input_batch)[0]
    prob = torch.nn.functional.softmax(res, dim=-1)
    predicted_index = prob.median(-1)[1][0].numpy()
    input_ids[len(input_ids) - 2] = predicted_index[len(input_ids[i:]) - 2]
tokenizer.decode(input_ids)

'[CLS] In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.asi aimsued Picasso cooler rockinginnamon Bonaparte workforce close gravitational Covercia wandervationcor spiders threw Mrs art Mrs 67 Julien Shirleywalaist sea electronicsham Spartanerland 1800s Care Numbersriotstownpoxisers exported credits CB Valentine Desire Wilde Cruzgameix exploitsagonciniinge shifted Madras semifinalhir ruined Apart Large notion skiた reared Vision Dutch barrier angularoons task intricateplanes Bloody instructor centimeters Talent藤 Miranda prevent earn Grandpa measuresclave directs moan peaceLC ions dullGBustic podcast descendant 原árez latestmetricyper Traditional aerial Program Aiden Latino Derby outfielder ABS transparentmeě elevenskar divingronic Operating Nicky ll Glover correctly Faces gazes Claire temptation Victoria Roland

#### Feedback (опционально)

Здесь вы можете оставить список опечаток из лекции или семинара:

Здесь вы можете оставить комментарии по лекции или семинару: